# Metaprogramming

Julia, igual que Lisp, representa al código como una estructura de datos en el propio lenguaje. La posibilidad de escribir código que genere y modifique código es lo que se entiende por "Metaprogramming".

Ilustraremos algunos conceptos siguiendo el [manual](http://docs.julialang.org/en/release-0.4/manual/metaprogramming/). 


Empecemos por decir lo que en algún sentido suena medio trivial, que es que cualquier línea de código, inicialmente se considera como una cadena:

In [1]:
one_plus_one = "1+1"

"1+1"

Lo que hace Julia internamente es convertir esa cadena en una *expresión*:

In [2]:
ex1 = parse(one_plus_one)

:(1 + 1)

In [3]:
typeof(ex1)

Expr

In [4]:
?Expr

search: Expr export nextprod expanduser expm exp2 exp expm1 exp10 expand



No documentation found.

**Summary:**

```julia
type Expr <: Any
```

**Fields:**

```julia
head :: Symbol
args :: Array{Any,1}
typ  :: Any
```


Claramente, un objeto tipo `Expr` tiene tres partes:

- Un `Symbol`, que se guarda en `head`:

In [5]:
ex1.head

:call

- Un vector (tipo `Any`) que contiene a los argumentos:

In [6]:
ex1.args

3-element Array{Any,1}:
  :+
 1  
 1  

- el tipo resultante de la expresión, que puede ser anotado por el usuario

In [7]:
ex1.typ

Any

Las expresiones también pueden ser escritas directamente, respetando el órden de los campos que definen la expresión:

In [8]:
ex2 = Expr(:call, :+, 1, 1)

:(1 + 1)

El punto importante aquí es que el código en Julia está representado internamente por expresiones escritas en Julia.

La función `dump()` da información sobre la expresión:

In [9]:
dump(ex2)

Expr 
  head: Symbol call
  args: Array(Any,(3,))
    1: Symbol +
    2: Int64 1
    3: Int64 1
  typ: Any


Expresiones más complicadas pueden ser construidas:

In [10]:
ex3 = parse("(4 + 4) / 2")

:((4 + 4) / 2)

In [11]:
dump(ex3)

Expr 
  head: Symbol call
  args: Array(Any,(3,))
    1: Symbol /
    2: Expr 
      head: Symbol call
      args: Array(Any,(3,))
        1: Symbol +
        2: Int64 4
        3: Int64 4
      typ: Any
    3: Int64 2
  typ: Any


In [12]:
Meta.show_sexpr(ex3)

(:call, :/, (:call, :+, 4, 4), 2)

Uno de los usos de `:` es crear símbolos; uno también puede usar `symbol()`:

In [13]:
:foo == symbol("foo")

true

In [14]:
symbol(:var,'_',"sym")

:var_sym

Otro uso de `:` es crear expresiones sin usar `Expr`; a esto se le llama *quoting*.

In [15]:
ex = :(a+b*c+1)

:(a + b * c + 1)

In [16]:
dump(ex)

Expr 
  head: Symbol call
  args: Array(Any,(4,))
    1: Symbol +
    2: Symbol a
    3: Expr 
      head: Symbol call
      args: Array(Any,(3,))
        1: Symbol *
        2: Symbol b
        3: Symbol c
      typ: Any
    4: Int64 1
  typ: Any


Lo interesante, es que uno puede cambiar *programáticamente* el contenido de una expresión. Por ejemplo, en lugar de $ex=a + b*c + 1$ quiero tener $ex=a + b*c + 2.1$:

In [17]:
Meta.show_sexpr(ex)

(:call, :+, :a, (:call, :*, :b, :c), 1)

Dado que los argumentos de una expresión están en Julia, uno puede modificarlos. Por ejemplo:

In [18]:
ex.args[4] = 2.1

2.1

In [19]:
Meta.show_sexpr(ex)

(:call, :+, :a, (:call, :*, :b, :c), 2.1)

Otra alternativa para definir expresiones complejas es usando el bloque `quote ... end`

In [20]:
# Lo siguiente es equivalente a:
# ex = :(x=1; y=2; x+y)
ex = quote
    x = 1
    y = 2
    x + y
end

quote  # In[20], line 4:
    x = 1 # In[20], line 5:
    y = 2 # In[20], line 6:
    x + y
end

In [21]:
Meta.show_sexpr(ex)

(:block,
  :( # In[20], line 4:),
  (:(=), :x, 1),
  :( # In[20], line 5:),
  (:(=), :y, 2),
  :( # In[20], line 6:),
  (:call, :+, :x, :y)
)

Para evaluar una expresión (de tipo `Expr`), usamos `eval()`. Respecto a esto, vale la pena notar que, mientras no se ha evaluado una expresión las variables internas no tienen ningún valor; sin embargo, una vez evaluada, los valores de las variables internas quedan definidas. Esto puede causar problemas, llamados *side effects*.

In [22]:
x

LoadError: LoadError: UndefVarError: x not defined
while loading In[22], in expression starting on line 1

In [23]:
y = 3

3

In [24]:
eval(ex)

3

In [25]:
x

1

In [26]:
y # y queda sobreescrita al evaluar la expresión !!

2

Uno incluso puede definir el valor de ciertas variables y usarlo como tal, o sustituirlo directamente:

In [27]:
z = 4

4

In [28]:
:(2*x+z)

:(2x + z)

In [29]:
eval(ans)

6

In [30]:
:(2*x+$z)

:(2x + 4)

In [31]:
:(2*x+$zz)

LoadError: LoadError: UndefVarError: zz not defined
while loading In[31], in expression starting on line 1

El punto es que uno puede definir funciones, que regresen expresiones, que pueden ser evaluadas

Un ejemplo más interesante es implementar los polinomios de Wilkinson, que se definen como:

\begin{equation}
    W_n(x) = \prod_{i=1}^n (x-i) = (x-1) (x-2) \cdots (x-n).
\end{equation}

Ciertamente este ejemplo se puede implementar a través de una recursión; implementémoslo generando código escrito por Julia.

In [32]:
nombre(n::Int) = symbol( string("W_", n) )

nombre (generic function with 1 method)

In [33]:
nombre(3)

:W_3

In [37]:
function wilkinson(n::Int)
    # Checo que `n` sea >= 1
    @assert n ≥ 1   # ≥ se obtiene con \ge<TAB>
    
    ex = :(x-1)
    for i = 2:n
        ex = :( ($ex) * ( x-$i) )
    end
    ex_ret = :( $(nombre(n))(x) = $ex )
    ex_ret
end

wilkinson (generic function with 1 method)

In [35]:
wilkinson(0)

LoadError: LoadError: AssertionError: n ≥ 1
while loading In[35], in expression starting on line 1

In [38]:
wilkinson(3)

:(W_3(x) = begin  # In[37], line 9:
            ((x - 1) * (x - 2)) * (x - 3)
        end)

In [39]:
eval(ans)

W_3 (generic function with 1 method)

In [40]:
W_3(2)

0

In [41]:
eval(wilkinson(4))

W_4 (generic function with 1 method)

In [42]:
W_4(2.5)

0.5625

**Ejercicio 1:** Implementen el factorial con metaprogramming.

## Macros

En varias ocasiones hemos usado ya algunas instrucciones que incluyen `@` antes, por ejemplo, `@time` o `@assert`. Este tipo de funciones son *macros*: Los macros son funciones cuyas entradas son *expresiones*, que en algún sentido son manipuladas y al final evaluadas.

In [43]:
macro simple_example(expr)
    @show expr   # this is another macro !
    return 0     # for simplicity
end

In [44]:
@simple_example(x+y)

expr = :(x + y)


0

In [45]:
@simple_example a = x + y

expr = :(a = x + y)


0

Cambiemos un poco el macro `@simple_example`; esta versión devuelve la expresión que es evaluada:

In [46]:
macro simple_example(expr)
    @show expr   # this is another macro !
    expr         # returns the input expression
end

In [47]:
x, y

(1,2)

In [48]:
@simple_example x + y

expr = :(x + y)


3

In [49]:
x = 3; y = 7;
@simple_example x + y

expr = :(x + y)


10

La gran sutileza de los macros, a diferencia de las funciones que hasta ahora hemos visto, es que permiten introducir y modificar código *antes* de que sea ejecutado, dado que los macros son ejecutados cuando el código es leído (*parse time*).

El siguiente ejemplo está tomado del manual.

In [50]:
macro twostep(arg)
    println("I execute at parse time. The argument is: ", arg)

    return :(println("I execute at runtime. The argument is: ", $arg))
end

In [51]:
ex = macroexpand( :(@twostep :(1, 2, 3)) );

I execute at parse time. The argument is: $(Expr(:quote, :((1,2,3))))


In [52]:
macroexpand( (:@simple_example x+y) )

expr = :(x + y)


:(x + y)

In [53]:
typeof(ex)

Expr

In [54]:
ex

:(println("I execute at runtime. The argument is: ",$(Expr(:copyast, :($(QuoteNode(:((1,2,3)))))))))

In [55]:
eval(ex)

I execute at runtime. The argument is: (1,2,3)


In [56]:
@twostep :(1, 2, 3)

I execute at parse time. The argument is: $(Expr(:quote, :((1,2,3))))
I execute at runtime. The argument is: (1,2,3)


## Generación de código

Arriba, con la definición `wilkinson()` de hecho definimos una función que genera código. Si bien ése no es el mejor ejemplo para esto, el punto es que a veces, excepto por pequeños cambios (por ejemplo, en los operadores), el código esencialmente se repite.


Como ejemplo, usaremos el tipo `AuDiff`, y sus métodos:

In [57]:
type AuDiff{T<:Real}
    fun :: T
    der :: T
end

AuDiff(a, b) = AuDiff(promote(a,b)...)

AuDiff{T<:Real}(a::T) = AuDiff(a, zero(T))

AuDiff{T<:Real}

In [58]:
import Base: convert

convert(::Type{AuDiff}, b::Real) = AuDiff(b)
convert{T<:Real, S<:Real}(::Type{AuDiff{T}}, b::S) = AuDiff(convert(T,b))
convert{T<:Real}(::Type{AuDiff{T}}, b::T) = AuDiff(b)
convert{T<:Real, S<:Real}(::Type{AuDiff{T}}, b::AuDiff{S}) = 
    AuDiff(convert(T,b.fun),convert(T,b.der))


convert (generic function with 541 methods)

In [59]:
import Base: promote_rule, promote

promote_rule{T<:Real}(::Type{AuDiff{T}}, ::Type{T}) = AuDiff{T}
promote_rule{T<:Real, S<:Real}(::Type{AuDiff{T}}, ::Type{S}) = AuDiff{promote_type(T, S)}

promote_rule (generic function with 126 methods)

In [60]:
import Base: +, -, *, /, ^

# Aquí definimos `+` y `-`
for op = (:+, :-)
    @eval ($op)(a::AuDiff, b::AuDiff) = AuDiff( ($op)(a.fun,b.fun), ($op)(a.der,b.der))
end

*(a::AuDiff, b::AuDiff) = AuDiff(a.fun*b.fun, a.fun*b.der+a.der*b.fun)

function /(a::AuDiff, b::AuDiff)
    nn = a.fun/b.fun
    dd = (a.der - nn*b.der)/b.fun
    return AuDiff(nn, dd)
end

^(a::AuDiff, n::Int) = AuDiff(a.fun^n, n * a.der * a.fun^(n-1))

^ (generic function with 48 methods)

In [61]:
sin(AuDiff(0.0,1.0))

LoadError: LoadError: MethodError: `sin` has no method matching sin(::AuDiff{Float64})
while loading In[61], in expression starting on line 1

In [62]:
import Base: exp, log, sin, cos, tan

tup_fn   = [:exp, :log, :sin, :cos, :tan]
tup_fder = [:exp, :(x -> 1/x), :(x -> cos(x)), :(x -> -sin(x)), :(x -> (sec(x))^2)]

for (fn, fnder) in zip(tup_fn, tup_fder)
    @eval ($fn)(a::AuDiff) = AuDiff( ($fn)(a.fun), a.der*($fnder)(a.fun) )
end

In [63]:
sin(AuDiff(0.0,1.0))

AuDiff{Float64}(0.0,1.0)